<a href="https://colab.research.google.com/github/Sourasky-DHLAB/Whisper/blob/main/Colab/Whisper_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="direction:rtl" dir="rtl">
    <img src="https://drive.google.com/uc?export=view&amp;id=15xQaGVrJej6UIPjfAB-QVWXLWVChPJSt"><br>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">    נכתב על ידי <a href="mailto:odedzarchia@tauex.tau.ac.il">עודד זרחיה</a> | <a href="https://github.com/Sourasky-DHLAB">Github</a><br>
    <a href="https://cenlib.tau.ac.il/">הספרייה המרכזית ע"ש סוראסקי</a> | <a href="https://tau.ac.il/">אוניברסיטת תל אביב</a>
</div></p>

<h1 id="-div-style-direction-rtl-dir-rtl-9-div-"><strong><div style="direction:rtl" dir="rtl">1. תמלול קבצי וידאו באמצעות Whisper</div></strong></h1>
<div style="direction:rtl" dir="rtl">
 <p style="text-align: right; direction: rtl; float: right;"><a href="https://openai.com/blog/whisper/">Whisper</a> הוא מודל לזיהוי דיבור מבית <a href="https://https://openai.com/">OpenAI</a> הזמין לציבור הרחב בקוד פתוח. מודל זה אומן על יותר מ-680 אלף שעות של שיחות באנגלית ובשפות רבות אחרות – בהן גם עברית וערבית.<br>מחברת זו תדריך אתכם בתמלול קטעי וידאו באמצעות Whisper. תהליך זה מורכב משלושה שלבים מרכזיים:</p>


<ol style="float:right;">
  <li>חילוץ שכבת האודיו מקובץ הוידאו.</li>
  <li>תמלול שכבת האודיו.</li>
  <li>בדיקת איכות (אופציונלי).</li></ol>

<div style="direction:rtl" dir="rtl">
<p style="text-align: right; direction: rtl; float: right;"> תוכלו להשתמש במחברת כפי שהיא כדי לאחסן את קבצי התמליל ב-Google Drive.
לתשומת לבכם - השימוש במחברת זו ובמודל של Whisper חופשי לחלוטין וללא שום עלות. בנוסף, אין הגבלה על אורך קטעי הוידאו/אודיו שניתן לתמלל באמצעות Whisper.</p>
</div>

# **<div style="direction:rtl" dir="rtl">2. בדוק את סוג המעבד הגרפי (GPU) </div>**
<div style="direction:rtl" dir="rtl">

סוג [המעבד הגרפי](https://https://he.wikipedia.org/wiki/%D7%9E%D7%A2%D7%91%D7%93_%D7%92%D7%A8%D7%A4%D7%99) **(GPU - Graphics Processing Unit)** שאתם מקבלים ב-Google Colab מגדיר את המהירות שבה קטעי האודיו יתומללו. ככל שמספר [הפלופס](https://https://he.wikipedia.org/wiki/FLOPS) **(FLOPS - Floating Point Operations Per Second, פעולות נקודות צפות לשנייה)** גבוה יותר, כך התמלול מהיר יותר. יחד עם זאת, גם המעבד הגרפי החלש ביותר ב-Colab מסוגל להריץ כל מודל של Whisper. יש לוודא כי בחרתם ב-GPU כמאיץ חומרה עבור מחברת זו (Runtime → Change runtime type → Hardware accelerator).
</div>

|  GPU   |  GPU RAM   | FP32 teraFLOPS |     Availability   |
|:------:|:----------:|:--------------:|:------------------:|
|  T4    |    16 GB   |       8.1      |         Free       |
| P100   |    16 GB   |      10.6      |      Colab Pro     |
| V100   |    16 GB   |      15.7      |  Colab Pro (Rare)  |

<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי לוודא את סוג המעבד הגרפי שהוקצה עבור מחברת זו. אפסו את ה-runtime של המחברת אם ברצונכם לקבל מעבד גרפי אחר (Runtime → Restart runtime).

In [ ]:
# check gpu type
!nvidia-smi -L
!nvidia-smi

# **<div style="direction:rtl" dir="rtl">3. התקנת ספריות </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי להוריד ולהתקין את הספריות הנחוצות לפעולות חילוץ האודיו,   התמלול ובדיקת האיכות:

1.   [whisper](https://https://github.com/openai/whisper) (עבור תמלול)
2.   [moviepy](https://https://pypi.org/project/moviepy/) (עבור חילוץ אודיו)
3.   [imageio](https://https://pypi.org/project/imageio/) (נדרש עבור moviepy)
4.   [jiwer](https://https://pypi.org/project/jiwer/) (עבור בדיקת WER)

In [ ]:
!pip install git+https://github.com/openai/whisper.git 
!pip install moviepy
!pip3 install imageio==2.4.1
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-8ig00pzu
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-8ig00pzu
  Resolved https://github.com/openai/whisper.git to commit 7858aa9c08d98f75575035ecd6481f462d66ca27
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230124-py3-none-any.whl size=1179424 sha256=b466dee4b79ec3e5560b46651550e396408bfa97d341a83f225af2e07b140f64
  Stored in directory: /tmp/pip-ephem-wheel-cache-3e0978rt/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built opena

# **<div style="direction:rtl" dir="rtl">4. ייבוא ספריות </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי לייבא את הספריות הנדרשות.

In [ ]:
import whisper
import moviepy.editor
from jiwer import wer
import os

# **<div style="direction:rtl" dir="rtl">5. חיבור ל-Google Drive </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי להתחבר ל-Google Drive האישי שלכם. כדי לראות את הקבצים שלכם פתחו את סייר הקבצים בתפריט השמאלי.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


# **<div style="direction:rtl" dir="rtl">6. הגדרת תיקיות </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי להגדיר את מיקום קבצי הוידאו, האודיו והתמליל:
1. קבצי וידאו - מחברת זו מניחה כי קבצי הוידאו לתמלול נמצאים ב-Google Drive תחת תיקיית `Whisper/Video`. ניתן לשנות הגדרה זו במשתנה `video_folder`.
2. קבצי אודיו - מחברת זו מניחה כי קבצי האודיו לתמלול נמצאים ב-Google Drive תחת תיקיית `Whisper/Audio`. ניתן לשנות הגדרה זו במשתנה `audio_folder`.
2. קבצי תמליל - מחברת זו מניחה כי קבצי הטקסט של התמלול יישמרו ב-Google Drive תחת תיקיית `Whisper/Transcriptions`. ניתן לשנות הגדרה זו במשתנה `transcription_folder`. במידה ואינה קיימת, תיקייה זו תיווצר באופן אוטומטי לאחר הרצת התא.


In [ ]:
# Assuming the video files are in a folder called "Video" under "Whisper"
video_folder = "/content/drive/MyDrive/Whisper/Video/"

# Assuming the audio files are in a folder called "Audio" under "Whisper"
audio_folder = "/content/drive/MyDrive/Whisper/Audio/"

# Assuming the text files will be placed in a folder called "Transcriptions" under "Whisper"
transcription_folder = "/content/drive/MyDrive/Whisper/Transcriptions/"

# Create "Transcriptions" folder if does not exist
if not os.path.exists(transcription_folder):
  os.makedirs(transcription_folder)

# **<div style="direction:rtl" dir="rtl">7. טעינת וידאו עם moviepy </div>**
<div style="direction:rtl" dir="rtl">

טענו קובץ וידאו מ-Google Drive באמצעות moviepy. שנו את הנתיב תחת משתנה `video` כדי לטעון את הקובץ שלכם (אתרו את הקובץ בסייר הקבצים, לחצו על הכפתור הימני בעכבר ← Copy path).

In [ ]:
video = moviepy.editor.VideoFileClip("/content/drive/MyDrive/Whisper/Video/netanyahu_liar_son_of_liar.mp4")

# **<div style="direction:rtl" dir="rtl">8. חילוץ שכבת אודיו </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי לחלץ את שכבת האודיו מקובץ הוידאו שלכם באמצעות moviepy וכדי לשמור אותה כקובץ נפרד ב-Google Drive שלכם. הקפידו לשנות את שם הקובץ בהתאם. 

In [ ]:
audio = video.audio
audio.write_audiofile("/content/drive/MyDrive/Whisper/Audio/netanyahu_liar_son_of_liar.mp3")

[MoviePy] Writing audio in /content/drive/MyDrive/Whisper/Audio/netanyahu_liar_son_of_liar.mp3


100%|██████████| 1883/1883 [00:03<00:00, 626.23it/s]

[MoviePy] Done.


# **<div style="direction:rtl" dir="rtl">9. טעינת מודל התמלול </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי לטעון את מודל התמלול המתאים.
יושם לב כי אנו משתמשים במודל הגדול (**large**), המדויק ביותר בשפה העברית. <br>
ישנם חמישה מודלים, ארבעה עם גרסאות באנגלית בלבד, המציעים פשרות שונות בין מהירות לדיוק. בטבלה להלן מוצגים שמות המודלים הזמינים יחד עם דרישות הזיכרון והמהירות היחסית שלהם:
</div>

|  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
|:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
|  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~1 GB     |      ~32x      |
|  base  |    74 M    |     `base.en`      |       `base`       |     ~1 GB     |      ~16x      |
| small  |   244 M    |     `small.en`     |      `small`       |     ~2 GB     |      ~6x       |
| medium |   769 M    |    `medium.en`     |      `medium`      |     ~5 GB     |      ~2x       |
| large  |   2870 M   |        N/A         |      `large`       |    ~10 GB     |       1x       |

<div style="direction:rtl" dir="rtl"><br>

במידה וברצונכם להתשמש במודל המדויק ביותר לעברית (**large**) עבור קבצים מרובים, מומלץ להריץ את המחברת בסביבה עם משאבי מחשוב מובטחים כדוגמת Google Colab Pro. 

In [ ]:
# load whisper model
model = whisper.load_model("large")
print(f"\033[1m Model loaded successfully")

100%|█████████████████████████████████████| 2.87G/2.87G [00:32<00:00, 95.8MiB/s]


 Model loaded successfully


# **<div style="direction:rtl" dir="rtl">10. קביעת שפה לתמלול </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי להגדיר את שפת התמלול כעברית. כדי להחליף את השפה שנו את ערך המשתנה `lang`.
<br>
ניתן להדפיס את רשימת השפות הנתמכת על ידי הרצת הפקודה`print(whisper.tokenizer.LANGUAGES)`בתא נפרד. 


In [ ]:
lang = 'he'

# **<div style="direction:rtl" dir="rtl">11. תמלול </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי לתמלל את קובץ האודיו בתיקיית Whisper/Audio. הקפידו לשנות את שם הקובץ.

In [ ]:
# set timer
import time
start_time = time.time()
result = model.transcribe("/content/drive/MyDrive/Whisper/Audio/netanyahu_liar_son_of_liar.mp3", verbose = False, language = lang)
print(f"\033[1m--- Transcription finished in  %s seconds ---" % (time.time() - start_time))

100%|██████████| 8536/8536 [00:30<00:00, 277.03frames/s]

--- Transcription finished in  32.942219257354736 seconds ---


# **<div style="direction:rtl" dir="rtl">12. שמירת התמלול </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי לשמור את התמלול ב-Google Drive שלכם כקובץ טקסט.

In [ ]:
with open('/content/drive/MyDrive/Whisper/Transcriptions/netanyahu_liar_son_of_liar.txt', 'w') as f:
    f.write(result["text"])
print(result["text"])

 אני מאמין שמערכת משפט חזקה ועצמאית היא זו שמאפשרת את קיומם של כל המוסדות האחרים לדמוקרטיה. אבל אני מבקש שתראו לי רודנות אחת, חברה לא דמוקרטית אחת, שבה קיימת מערכת משפט חזקה ועצמאית. אין דבר כזה. בכל מקום שבו אין מערכת משפט חזקה ועצמאית, הזכויות אינן יכולות להיות מוגנות. למעשה, ההבדל בין המדינות שבהן יש זכויות על הנייר ובין מדינות שבהן יש זכויות בפועל, ההבדל הזה הוא מערכת משפט חזקה ועצמאית. זו הסיבה שאני עושה, ואמשיך לעשות כל שביכולתי, כדי לשמור על מערכת משפט חזקה ועצמאית. רק בחודשים האחרונים גנזתי כל חוק שאיים לפגוע בעצמאות המערכת מהניסיון לקיים שימויים לשופטים בכנסת, דרך הגבלת עתירות לבית המשפט ועד לשינוי ירכב הוועדה לבחירת שוטים. אני אמשיך לפעול כך. ובכל פעם שיגיע לשולחני חוק שעלול לפגוע בעצמאות בתי המשפט בישראל, אנחנו נוריד אותו.


# **<div style="direction:rtl" dir="rtl">13. בדיקת WER (אופציונלי) </div>**
<div style="direction:rtl" dir="rtl">

הרצת התאים הבאים תאפשר לכם להעריך את הדיוק של התמלול של Whisper במספר שלבים, וזאת במידה ויש ברשותכם את קובץ התמלול המקורי:



1.   נירמול פלט התמלול של Whisper וקובץ התמלול המקורי ([Ground Truth](https://https://en.wikipedia.org/wiki/Ground_truth)) - הסרת סימני פיסוק, רווחים מיותרים, קפיטליזציה וכו'.
2.   השוואה בין שני הקבצים וחישוב אחוז השגיאות בהתאם למדד [Word Error Rate](https://https://en.wikipedia.org/wiki/Word_error_rate) - באמצעות ספריית jiwer.

הפלט המתקבל מוצג באחוזים ומצביע על אחוז המילים השגויות בקובץ התמלול, וזאת בהשוואה לקובץ המקור. כלומר, במידה ואחוז השגיאות עומד על כשלושה אחוזים, אזי מתוך 100 מילים יש 3 מילים שגויות.

In [ ]:
# normalize result
import string
heb_transcription_norm = open('/content/drive/MyDrive/Whisper/Transcriptions/netanyahu_liar_son_of_liar.txt', 'r').read().lower().translate(str.maketrans('', '', string.punctuation))
print(heb_transcription_norm)

 אני מאמין שמערכת משפט חזקה ועצמאית היא זו שמאפשרת את קיומם של כל המוסדות האחרים לדמוקרטיה אבל אני מבקש שתראו לי רודנות אחת חברה לא דמוקרטית אחת שבה קיימת מערכת משפט חזקה ועצמאית אין דבר כזה בכל מקום שבו אין מערכת משפט חזקה ועצמאית הזכויות אינן יכולות להיות מוגנות למעשה ההבדל בין המדינות שבהן יש זכויות על הנייר ובין מדינות שבהן יש זכויות בפועל ההבדל הזה הוא מערכת משפט חזקה ועצמאית זו הסיבה שאני עושה ואמשיך לעשות כל שביכולתי כדי לשמור על מערכת משפט חזקה ועצמאית רק בחודשים האחרונים גנזתי כל חוק שאיים לפגוע בעצמאות המערכת מהניסיון לקיים שימויים לשופטים בכנסת דרך הגבלת עתירות לבית המשפט ועד לשינוי ירכב הוועדה לבחירת שוטים אני אמשיך לפעול כך ובכל פעם שיגיע לשולחני חוק שעלול לפגוע בעצמאות בתי המשפט בישראל אנחנו נוריד אותו


In [ ]:
# normalize ground truth
heb_ground_truth_norm = open('/content/drive/MyDrive/Whisper/GT/netanyahu_ground_truth.txt', 'r').read().lower().strip().translate(str.maketrans('', '', string.punctuation))
print(heb_ground_truth_norm)

אני מאמין שמערכת משפט חזקה ועצמאית היא זו שמאפשרת את קיומם של כל המוסדות האחרים בדמוקרטיה אבל אני מבקש שתראו לי רודנות אחת חברה לא דמוקרטית אחת שבה קיימת מערכת משפט חזקה ועצמאית אין דבר כזה בכל מקום שבו אין מערכת משפט חזקה ועצמאית הזכויות אינן יכולות להיות מוגנות למעשה ההבדל בין המדינות שבהן יש זכויות על הנייר ובין מדינות שבהן יש זכויות בפועל ההבדל הזה הוא מערכת משפט חזקה ועצמאית וזוהי הסיבה שאני עושה ואמשיך לעשות כל שביכולתי כדי לשמור על מערכת משפט חזקה ועצמאית רק בחודשים האחרונים גנזתי כל חוק שאיים לפגוע בעצמאות המערכת מהניסיון לקיים שימועים לשופטים בכנסת דרך הגבלת עתירות לבית המשפט ועד לשינוי הרכב הוועדה לבחירת שוטים אני אמשיך לפעול כך ובכל פעם שיגיע לשולחני חוק שעלול לפגוע בעצמאות בתי המשפט בישראל אנחנו נוריד אותו


In [ ]:
# calculate WER
ground_truth = heb_ground_truth_norm
hypothesis = heb_transcription_norm
error = wer(ground_truth, hypothesis)
error_percentage = "{:.0%}".format(error)
print(error_percentage)

3%
